# faab-run-loop


In [ ]:
import torch 
from pybela import Streamer
import json
import asyncio
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os 

from utils import get_device, load_config, load_model, get_all_run_ids, load_train_loss

device = get_device()

path ="models/trained"

## run-loop testing

In [3]:
vars = ['gFaabSensor_1', 'gFaabSensor_2', 'gFaabSensor_3', 'gFaabSensor_4', 'gFaabSensor_5', 'gFaabSensor_6', 'gFaabSensor_7', 'gFaabSensor_8']
streamer = Streamer()
streamer.connect()

Connection successful


1

In [ ]:
_id = "02emtq9d"
model = load_model(_id, path=path)
seq_len = 512

async def callback(block):
    _raw_data_tensor = torch.stack([torch.FloatTensor(buffer["buffer"]["data"]) for buffer in block]) # num_features, 1024
    inputs = _raw_data_tensor.unfold(1, seq_len, seq_len).permute(1,2,0) # n, seq_len, num_features  
    
    for _input in inputs:
        out = model(_input.to(device)).permute(1,0) # num_features, seq_len
        for idx,feature in enumerate(out):
            streamer.send_buffer(idx, 'f', seq_len, feature.tolist(), verbose=True)
            
streamer.start_streaming(vars, on_block_callback=callback)
asyncio.run(asyncio.sleep(10))
streamer.stop_streaming()



## models params

In [4]:
run_ids = get_all_run_ids(path=path)
params = ["ff_size", "num_heads", "num_layers", "learning_rate"]
_id_config = {}

for _id in run_ids:
    _id_config[_id] = {key: load_config(_id, path=path)[key] for key in params}
    
df = pd.DataFrame.from_dict(_id_config, orient='index')
df.head(10)    

,ff_size,num_heads,num_layers,learning_rate
abor4fs6,32,2,1,0.000516
qcamd717,256,2,8,0.000112
3r3rbr3q,256,1,7,0.000168
ggnxfugo,256,1,7,0.000842
qba7ejlo,16,4,3,0.000149
2kqwwsar,128,4,6,0.000466
ejdzc3ym,64,2,7,0.000850
24jf630m,64,4,7,0.000105
jeixb37n,8,4,6,0.000838
u0ahkjvo,32,4,7,0.000797


### scale to values between 0 and 1

In [5]:
ranges = {
    "ff_size": [8, 16, 32, 64, 128, 256],
    "num_heads": [1, 2, 4],
    "num_layers": [1, 2, 3, 4, 5, 6, 7, 8]
}

mapped_ranges = {}
for key in ranges:
    mapped_ranges[key] = {value: idx /(len(ranges[key])-1) for idx, value in enumerate(ranges[key])}

ranges, mapped_ranges

({'ff_size': [8, 16, 32, 64, 128, 256],
  'num_heads': [1, 2, 4],
  'num_layers': [1, 2, 3, 4, 5, 6, 7, 8]},
 {'ff_size': {8: 0.0, 16: 0.2, 32: 0.4, 64: 0.6, 128: 0.8, 256: 1.0},
  'num_heads': {1: 0.0, 2: 0.5, 4: 1.0},
  'num_layers': {1: 0.0,
   2: 0.14285714285714285,
   3: 0.2857142857142857,
   4: 0.42857142857142855,
   5: 0.5714285714285714,
   6: 0.7142857142857143,
   7: 0.8571428571428571,
   8: 1.0}})

In [6]:
column_functions = {
    "ff_size": lambda x: mapped_ranges["ff_size"][x],
    "num_heads": lambda x: mapped_ranges["num_heads"][x],
    "num_layers": lambda x:mapped_ranges["num_layers"][x],
    "learning_rate": lambda x: x*1000
}


# Apply a different function to each column
df_scaled = df.copy()
for column, func in column_functions.items():
    if column in df_scaled.columns:
        df_scaled[column] = df_scaled[column].apply(func)
        
df_scaled.head(10)

,ff_size,num_heads,num_layers,learning_rate
abor4fs6,0.4,0.5,0.000000,0.516246
qcamd717,1.0,0.5,1.000000,0.112486
3r3rbr3q,1.0,0.0,0.857143,0.168206
ggnxfugo,1.0,0.0,0.857143,0.841713
qba7ejlo,0.2,1.0,0.285714,0.149064
2kqwwsar,0.8,1.0,0.714286,0.465967
ejdzc3ym,0.6,0.5,0.857143,0.849721
24jf630m,0.6,1.0,0.857143,0.105182
jeixb37n,0.0,1.0,0.714286,0.837599
u0ahkjvo,0.4,1.0,0.857143,0.797156


In [ ]:
scaled_model_coordinates = df_scaled.to_numpy()
scaled_model_coordinates.shape

In [ ]:
from utils import get_scaled_params
scaled_model_coordinates = get_scaled_params()

In [ ]:
from utils import find_closest_model
import numpy as np
vector = np.array([0.1, 0.2, 0.3, 0.4])  # Replace with your actual vector

find_closest_model(vector, scaled_model_coordinates)

In [ ]:
list(scaled_model_coordinates.values())

test the scaling (assumes model output to be between 0 and 1)

In [ ]:
vector = np.array([0.1, 0.2, 0.3, 0.4])  # Replace with your actual vector

# Calculate the Euclidean distances
distances = np.linalg.norm(scaled_model_coordinates - vector, axis=1)

# Find the index of the row with the smallest distance
closest_row_index = np.argmin(distances)

# Closest row
closest_row = scaled_model_coordinates[closest_row_index]

print(closest_row)

In [ ]:
# Create a figure and a grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(20, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, column in enumerate(df_scaled.columns):
    sns.histplot(df_scaled[column], ax=axes[i])
    axes[i].set_title(f'Distribution of Values for {column}')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequency')

# Hide any unused axes if the number of columns is odd
if len(df_scaled.columns) % 2 != 0:
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

In [2]:
run_ids = get_all_run_ids(path=path)
_train_losses = {}
for _id in run_ids:
    _train_losses[_id] =  load_train_loss(_id, path=path)

In [3]:
_train_losses

{'abor4fs6': 0.01330749,
 'qcamd717': 0.01550733,
 '3r3rbr3q': 0.01433495,
 'ggnxfugo': 0.01649893,
 'qba7ejlo': 0.01445403,
 '2kqwwsar': 0.01324307,
 'ejdzc3ym': 0.01351297,
 '24jf630m': 0.01406082,
 'jeixb37n': 0.01283396,
 'u0ahkjvo': 0.01294638,
 '8wqxa1om': 0.01297873,
 'altyemr7': 0.01383731,
 'a3nrg221': 0.01332444,
 'xoge2peb': 0.01296615,
 '8wlok5de': 0.01374957,
 'yzxtaqw4': 0.01413367,
 'jsrbrjo7': 0.01327488,
 'ezzoxwy9': 0.01318688,
 '5li6ry52': 0.01329148,
 'b7oluf4m': 0.01310377,
 'dsgge1ug': 0.01441944,
 'kzj2k4h6': 0.01321384,
 'j2ke14to': 0.01361384,
 'lt03ebc6': 0.01352065,
 'hnqlho8c': 0.01392986,
 'va757ku0': 0.01379304,
 'h0o65m8s': 0.01337427,
 'kyi2wb9t': 0.01317669,
 'o0oxwb1j': 0.0148841,
 'h8b4mag0': 0.01450323,
 'qet3z9w4': 0.01398599,
 '2le0nl3f': 0.01305585,
 '9f122a6h': 0.0140853,
 '5ll03r5l': 0.01365658,
 'b97z5v31': 0.01344458,
 'hj6qimek': 0.01331878,
 'j4nx4ad2': 0.01425902,
 'b2kr7jlj': 0.01469418,
 'eleqs608': 0.01406533,
 '02emtq9d': 0.02764707,
 '

In [10]:
models_ordered_by_asc_loss = list(dict(sorted(_train_losses.items(), key=lambda item: item[1])).keys())


In [11]:
filename = path + "/models_ordered_by_asc_loss.json"

with open(filename, 'w') as file:
    json.dump(models_ordered_by_asc_loss, file)

In [ ]:
for _id in run_ids:
    _id_config[_id] = {key: load_config(_id, path=path)[key] for key in params}
    
df = pd.DataFrame.from_dict(_id_config, orient='index')
df.head(10)   